In [73]:
# data
import nfl_data_py as nfl

# data loading and plotting
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# models
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier, plot_importance

# interpretation
import shap
from interpret import show

# pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, r2_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, nan_euclidean_distances

pd.set_option('display.max_columns', None)  # None zeigt alle Spalten
pd.set_option('display.max_rows', None)  # Alle Zeilen anzeigen, vorsichtig bei großen DataFrames
pd.set_option('display.width', 1000)  # Breite anpassen

In [115]:
df_weekly = nfl.import_weekly_data([2024])
df_weekly = df_weekly[(df_weekly['season_type'] == 'REG') & (df_weekly['position'].isin(['QB', 'WR', 'RB', 'TE']))]
df_ids = nfl.import_ids()

df_ids = df_ids.rename(columns={'pfr_id': 'pfr_player_id'})

df_weekly = pd.merge(df_weekly, df_ids[['gsis_id', 'pfr_player_id', 'draft_pick', 'draft_year']], left_on = 'player_id', right_on= 'gsis_id', how = 'inner')



Downcasting floats.


In [116]:
df_weekly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5162 entries, 0 to 5161
Data columns (total 57 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player_id                    5162 non-null   object 
 1   player_name                  5162 non-null   object 
 2   player_display_name          5162 non-null   object 
 3   position                     5162 non-null   object 
 4   position_group               5162 non-null   object 
 5   headshot_url                 5162 non-null   object 
 6   recent_team                  5162 non-null   object 
 7   season                       5162 non-null   int32  
 8   week                         5162 non-null   int32  
 9   season_type                  5162 non-null   object 
 10  opponent_team                5162 non-null   object 
 11  completions                  5162 non-null   int32  
 12  attempts                     5162 non-null   int32  
 13  passing_yards     

In [104]:
df_ids.head(20)

,height,nfl_id,rotoworld_id,db_season,fantasy_data_id,pff_id,weight,draft_ovr,rotowire_id,stats_global_id,pfr_player_id,college,cfbref_id,merge_name,age,name,cbs_id,sportradar_id,swish_id,position,team,draft_pick,ktc_id,fleaflicker_id,birthdate,fantasypros_id,espn_id,stats_id,player_id,draft_year,yahoo_id,sleeper_id,mfl_id,draft_round,twitter_username
0,73.0,NaN,NaN,2024,NaN,NaN,215.0,1.0,17695.0,0.0,WillCa03,USC,caleb-williams-3,caleb williams,23.1,Caleb Williams,26695775.0,49d4b627-3f19-49a8-ae33-f539e0fb1f88,1286445.0,QB,CHI,1.0,1550.0,NaN,2001-11-18,23084.0,4431611.0,40900.0,00-0039918,2024.0,40900.0,11560.0,16579,1.0,NaN
1,76.0,NaN,NaN,2024,NaN,NaN,225.0,3.0,17673.0,0.0,MayeDr00,North Carolina,drake-maye-1,drake maye,22.3,Drake Maye,26695757.0,d0c0630e-8925-4b81-a32b-e4146d95f01f,1286803.0,QB,NEP,3.0,1551.0,NaN,2002-08-30,23046.0,4431452.0,40881.0,00-0039851,2024.0,40881.0,11564.0,16580,1.0,NaN
2,76.0,NaN,NaN,2024,NaN,NaN,210.0,2.0,17692.0,0.0,DaniJa02,LSU,jayden-daniels-1,jayden daniels,24.0,Jayden Daniels,3123337.0,bab8c65a-9fff-410d-8e34-dc2f636d890e,1161228.0,QB,WAS,2.0,1552.0,NaN,2000-12-18,22902.0,4426348.0,40896.0,00-0039910,2024.0,40896.0,11566.0,16581,1.0,NaN
3,74.0,NaN,NaN,2024,NaN,NaN,217.0,12.0,16995.0,0.0,NixxBo00,Oregon,bo-nix-1,bo nix,24.9,Bo Nix,3121193.0,f056c8ed-6ee5-492c-af11-3f7e3d97195c,1161899.0,QB,DEN,12.0,1553.0,NaN,2000-02-25,22910.0,4426338.0,40875.0,00-0039732,2024.0,40875.0,11563.0,16582,1.0,NaN
4,74.0,NaN,NaN,2024,NaN,NaN,220.0,8.0,17700.0,0.0,PeniMi00,Washington,michael-penix-jr-1,michael penix,24.7,Michael Penix Jr.,2963496.0,6499a161-b221-4591-b122-d99ece1be64b,1107478.0,QB,ATL,8.0,1554.0,NaN,2000-05-08,22973.0,4360423.0,40889.0,00-0039917,2024.0,40889.0,11559.0,16583,1.0,NaN
5,75.0,NaN,NaN,2024,NaN,NaN,219.0,10.0,17689.0,0.0,McCaJJ00,Michigan,jj-mccarthy-1,jj mccarthy,22.0,J.J. McCarthy,26695214.0,587cc3c3-aeda-4ac7-8803-a7e4db195534,1287599.0,QB,MIN,10.0,1555.0,NaN,2003-01-20,23018.0,4433970.0,40905.0,00-0039923,2024.0,40905.0,11565.0,16584,1.0,NaN
6,77.0,NaN,NaN,2024,NaN,NaN,246.0,193.0,17830.0,0.0,MiltJo00,Tennessee,joe-milton-2,joe milton,24.8,Joe Milton,2962193.0,74043480-24ef-4c26-99d4-18f43d5b41d3,1112338.0,QB,NEP,17.0,1556.0,NaN,2000-03-06,25967.0,4360698.0,41060.0,00-0039398,2024.0,41060.0,11557.0,16585,6.0,NaN
7,72.0,NaN,NaN,2024,NaN,NaN,211.0,150.0,17782.0,0.0,RattSp00,South Carolina,spencer-rattler-1,spencer rattler,24.3,Spencer Rattler,3125863.0,e0eecf78-86b2-4ee8-8ac9-d7713a138765,1174752.0,QB,NOS,15.0,1557.0,NaN,2000-09-28,23081.0,4426339.0,41043.0,00-0039376,2024.0,41043.0,11562.0,16586,5.0,NaN
8,77.0,NaN,NaN,2024,NaN,NaN,306.0,NaN,17664.0,0.0,NaN,Florida,NaN,sam hartman,26.2,Sam Hartman,NaN,007971f0-f012-11ed-833c-ebfbb713adc7,1117351.0,QB,FA,NaN,NaN,NaN,1998-10-15,NaN,4361994.0,41238.0,00-0039677,2024.0,NaN,11376.0,16587,NaN,NaN
9,74.0,NaN,NaN,2024,NaN,NaN,217.0,245.0,16996.0,0.0,PratMi00,Tulane,michael-pratt-1,michael pratt,23.3,Michael Pratt,3163083.0,da5f9694-1b77-4f6e-acea-6fc76145f79a,1214318.0,QB,TBB,25.0,NaN,NaN,2001-09-30,25226.0,4685039.0,41130.0,00-0039238,2024.0,41130.0,11561.0,16588,7.0,NaN


In [99]:
df_draft.head()

,season,round,pick,team,gsis_id,pfr_player_id,cfb_player_id,pfr_player_name,hof,position,category,side,college,age,to,allpro,probowls,seasons_started,w_av,car_av,dr_av,games,pass_completions,pass_attempts,pass_yards,pass_tds,pass_ints,rush_atts,rush_yards,rush_tds,receptions,rec_yards,rec_tds,def_solo_tackles,def_ints,def_sacks
12156,2024,1,1,CHI,None,WillCa03,caleb-williams-3,Caleb Williams,False,QB,QB,O,USC,22.0,2024.0,0,0,0,0.0,NaN,0.0,1.0,14.0,29.0,93.0,0.0,0.0,5.0,15.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
12157,2024,1,2,WAS,None,DaniJa02,jayden-daniels-1,Jayden Daniels,False,QB,QB,O,LSU,23.0,2024.0,0,0,0,0.0,NaN,0.0,1.0,17.0,24.0,184.0,0.0,0.0,16.0,88.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN
12158,2024,1,3,NWE,None,MayeDr00,drake-maye-1,Drake Maye,False,QB,QB,O,North Carolina,22.0,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12159,2024,1,4,ARI,None,HarrMa09,marvin-harrison-jr-1,Marvin Harrison Jr.,False,WR,WR,O,Ohio St.,22.0,2024.0,0,0,0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,NaN,NaN,NaN
12160,2024,1,5,LAC,None,AltxJo01,joe-alt-1,Joe Alt,False,OL,OL,O,Notre Dame,21.0,2024.0,0,0,0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN


In [100]:
df_weekly[df_weekly['pfr_player_id']=='DaniJa02']

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,opponent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,pfr_player_id
5124,00-0039910,J.Daniels,Jayden Daniels,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,WAS,2024,1,REG,TB,17,24,184.0,0,0.0,2.0,23.0,1,0,125.0,154.0,7.0,-0.791879,0,1.472000,0.116929,16,88.0,2,2.0,0.0,8.0,8.706162,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,28.160000,28.160000,DaniJa02
5125,00-0039910,J.Daniels,Jayden Daniels,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,WAS,2024,2,REG,NYG,23,29,226.0,0,0.0,5.0,16.0,0,0,122.0,142.0,11.0,4.707816,0,1.852459,0.154793,10,44.0,0,0.0,0.0,2.0,1.690625,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,13.440000,13.440000,DaniJa02
5126,00-0039910,J.Daniels,Jayden Daniels,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,WAS,2024,3,REG,CIN,21,23,254.0,2,0.0,2.0,6.0,0,0,215.0,98.0,13.0,21.611946,0,1.181395,0.451954,12,39.0,1,0.0,0.0,5.0,3.295520,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,28.059999,28.059999,DaniJa02
5127,00-0039910,J.Daniels,Jayden Daniels,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,WAS,2024,4,REG,ARI,26,30,233.0,1,1.0,0.0,-0.0,0,0,174.0,122.0,14.0,13.108258,1,1.339080,0.281474,8,47.0,1,0.0,0.0,3.0,2.592485,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,24.020000,24.020000,DaniJa02
5128,00-0039910,J.Daniels,Jayden Daniels,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,WAS,2024,5,REG,CLE,14,25,238.0,1,1.0,3.0,17.0,0,0,301.0,90.0,8.0,2.378711,0,0.790698,0.067045,11,82.0,0,0.0,0.0,4.0,4.613883,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,19.719999,19.719999,DaniJa02
5129,00-0039910,J.Daniels,Jayden Daniels,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,WAS,2024,6,REG,BAL,24,35,269.0,2,0.0,3.0,16.0,0,0,276.0,101.0,16.0,9.493135,0,0.974638,0.181458,6,22.0,0,0.0,0.0,1.0,-0.185189,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,20.959999,20.959999,DaniJa02
5130,00-0039910,J.Daniels,Jayden Daniels,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,WAS,2024,7,REG,CAR,2,2,6.0,0,0.0,0.0,-0.0,0,0,-3.0,9.0,0.0,-1.239996,0,0.000000,NaN,3,50.0,0,0.0,0.0,1.0,1.964500,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,5.240000,5.240000,DaniJa02
5131,00-0039910,J.Daniels,Jayden Daniels,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,WAS,2024,8,REG,CHI,21,38,326.0,1,0.0,2.0,13.0,0,0,457.0,102.0,15.0,6.438087,0,0.713348,0.106203,8,52.0,0,0.0,0.0,2.0,4.174802,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,22.240000,22.240000,DaniJa02
5132,00-0039910,J.Daniels,Jayden Daniels,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,WAS,2024,9,REG,NYG,15,22,209.0,2,0.0,0.0,-0.0,0,0,132.0,107.0,8.0,9.809720,0,1.583333,0.282466,8,35.0,0,0.0,0.0,3.0,-2.090221,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,19.860001,19.860001,DaniJa02
5133,00-0039910,J.Daniels,Jayden Daniels,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,WAS,2024,10,REG,PIT,17,34,202.0,0,0.0,3.0,20.0,0,0,216.0,87.0,7.0,-9.261981,0,0.935185,-0.045807,3,5.0,0,0.0,0.0,0.0,-1.106103,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,8.580000,8.580000,DaniJa02


In [102]:
df_weekly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5227 entries, 0 to 5226
Data columns (total 54 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   player_id                    5227 non-null   object 
 1   player_name                  5227 non-null   object 
 2   player_display_name          5227 non-null   object 
 3   position                     5227 non-null   object 
 4   position_group               5227 non-null   object 
 5   headshot_url                 5227 non-null   object 
 6   recent_team                  5227 non-null   object 
 7   season                       5227 non-null   int32  
 8   week                         5227 non-null   int32  
 9   season_type                  5227 non-null   object 
 10  opponent_team                5227 non-null   object 
 11  completions                  5227 non-null   int32  
 12  attempts                     5227 non-null   int32  
 13  passing_yards     

In [79]:
df_draft['pfr_player_id'].nunique()

7388

In [84]:
df_weekly.head(50)

,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,opponent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,pfr_player_id,pick,draft_season
0,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NE,2018,1,REG,HOU,26,39,277.0,3,1.0,2.0,10.0,0,0,320.0,93.0,16.0,8.182894,0,0.865625,0.150320,1,2.0,0,0.0,0.0,1.0,1.837627,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,21.280001,21.280001,BradTo00,199.0,2000.0
1,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NE,2018,2,REG,JAX,24,35,234.0,2,0.0,2.0,14.0,1,1,213.0,136.0,12.0,4.945607,0,1.098592,0.128772,3,10.0,0,0.0,0.0,2.0,2.328974,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,16.360001,16.360001,BradTo00,199.0,2000.0
2,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NE,2018,3,REG,DET,14,26,133.0,1,1.0,2.0,13.0,0,0,333.0,48.0,4.0,-10.869923,0,0.399399,-0.008199,1,2.0,0,0.0,0.0,1.0,-0.245843,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,7.520000,7.520000,BradTo00,199.0,2000.0
3,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NE,2018,4,REG,MIA,23,35,274.0,3,2.0,0.0,-0.0,0,0,279.0,181.0,13.0,11.914713,0,0.982079,0.215751,0,0.0,0,0.0,0.0,0.0,NaN,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,18.959999,18.959999,BradTo00,199.0,2000.0
4,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NE,2018,5,REG,IND,34,44,341.0,3,2.0,0.0,-0.0,0,0,285.0,167.0,18.0,9.879805,0,1.196491,0.166761,3,-1.0,1,0.0,0.0,1.0,-0.820811,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,27.540001,27.540001,BradTo00,199.0,2000.0
5,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NE,2018,6,REG,KC,24,35,340.0,1,0.0,2.0,13.0,2,1,323.0,172.0,14.0,2.107192,0,1.052632,0.142699,3,2.0,1,0.0,0.0,1.0,1.886154,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,21.799999,21.799999,BradTo00,199.0,2000.0
6,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NE,2018,7,REG,CHI,25,36,277.0,3,1.0,1.0,4.0,0,0,235.0,132.0,15.0,14.705606,0,1.178723,0.191370,1,6.0,0,0.0,0.0,0.0,-0.371460,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,21.680000,21.680000,BradTo00,199.0,2000.0
7,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NE,2018,8,REG,BUF,29,45,324.0,0,0.0,2.0,13.0,1,0,343.0,132.0,15.0,8.506986,0,0.944606,0.122807,1,8.0,0,0.0,0.0,1.0,1.439446,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,13.760000,13.760000,BradTo00,199.0,2000.0
8,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NE,2018,9,REG,GB,22,35,294.0,1,0.0,2.0,21.0,0,0,266.0,159.0,12.0,0.211650,0,1.105263,0.061459,5,-1.0,0,0.0,0.0,1.0,-2.493572,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,15.660000,15.660000,BradTo00,199.0,2000.0
9,00-0019596,T.Brady,Tom Brady,QB,QB,https://static.www.nfl.com/image/private/f_aut...,NE,2018,10,REG,TEN,21,41,254.0,0,0.0,3.0,23.0,0,0,354.0,118.0,12.0,-3.916946,0,0.717514,-0.042354,1,0.0,0,0.0,0.0,0.0,-1.463562,0,1,1,6.0,0,0.0,0.0,0.0,6.0,0.0,-0.220586,0,0.000000,0.025000,0.000000,0.037500,0.0,10.760000,11.760000,BradTo00,199.0,2000.0


In [72]:
df = df[df['club_code'] == 'CIN']
df = df[df['position'] == 'QB']

KeyError: 'club_code'

In [ ]:
df.head(40)

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name
3510,2023,CIN,1.0,REG,1,Burrow,Joe,Joe,Offense,00-0036442,9,QB,BUR679731,QB,Joe Burrow
3511,2023,CIN,1.0,REG,2,Browning,Jake,Jake,Offense,00-0035100,6,QB,BRO817047,QB,Jake Browning
3573,2023,CIN,2.0,REG,1,Burrow,Joe,Joe,Offense,00-0036442,9,QB,BUR679731,QB,Joe Burrow
3574,2023,CIN,2.0,REG,2,Browning,Jake,Jake,Offense,00-0035100,6,QB,BRO817047,QB,Jake Browning
3659,2023,CIN,3.0,REG,2,Browning,Jake,Jake,Offense,00-0035100,6,QB,BRO817047,QB,Jake Browning
3680,2023,CIN,3.0,REG,1,Burrow,Joe,Joe,Offense,00-0036442,9,QB,BUR679731,QB,Joe Burrow
3726,2023,CIN,4.0,REG,1,Burrow,Joe,Joe,Offense,00-0036442,9,QB,BUR679731,QB,Joe Burrow
3727,2023,CIN,4.0,REG,2,Browning,Jake,Jake,Offense,00-0035100,6,QB,BRO817047,QB,Jake Browning
3786,2023,CIN,5.0,REG,1,Burrow,Joe,Joe,Offense,00-0036442,9,QB,BUR679731,QB,Joe Burrow
3788,2023,CIN,5.0,REG,2,Browning,Jake,Jake,Offense,00-0035100,6,QB,BRO817047,QB,Jake Browning
